In [ ]:
# Скачиваем библиотеку
!pip install mne

In [ ]:
# Подгружаем нужные нам библиотеки и их модули
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew , variation
from scipy.signal import welch

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)


In [ ]:
# прописываем путь к папке с файлами на гугл диске
path='EEG_files/'

In [ ]:
#загрузка сырого файла
ID='NP112212'
name=f'{path}{ID}_RT.vhdr' # остальные файлы подтянет сам
raw = mne.io.read_raw_brainvision(name,
                                  eog=('HEOGL', 'HEOGR', 'VEOGb'),
                                  misc='auto', scale=1.0,
                                  preload=True, verbose=None
                                 )

In [ ]:
raw.info #выводим информацию о файле

In [ ]:
raw.ch_names #Можем посмотреть названия каналов

In [ ]:
#удаляем лишние каналы - микрофон, фотодатчик (система не знает их точных координат)
raw.drop_channels(['Photo', 'micr']) # по умолчанию переписывает исходный объект

In [ ]:
#Смотрим располажение сенсеров
mne.viz.plot_sensors(raw.info, kind='topomap', show_names=True)
plt.show()

In [ ]:
raw.plot(n_channels = len(raw.ch_names), duration=20.0, start=10.0) # почему-то не переходит в интерактивный режим - может, нужна дополнительная библиотека типа QT?
plt.show()

In [ ]:
# фильтруем данные
raw.filter(0.1, 30, method = 'fir')
raw.plot(n_channels = len(raw.ch_names), duration=20.0, start=10.0)
plt.show()

In [ ]:
raw.plot(n_channels = len(raw.ch_names), duration=20.0, start=10.0, scalings='auto')
plt.show()

In [ ]:
len(raw)

In [ ]:
# for i in range (10, len(raw), 80):
#   raw.plot(n_channels = len(raw.ch_names), duration=30.0, start=i, scalings='auto')

In [ ]:
# задаем плохие каналы
raw.info['bads']=['T8']

In [ ]:
raw.info

In [ ]:
# интерполируем плохие каналы
raw.interpolate_bads()

In [ ]:
raw_ica = raw.copy()
raw_ica.filter(1, 30) #другой диапазон фильтрации для лучшей работы ICA
# созданим эпохи на основе записи
epochs_r = mne.make_fixed_length_epochs(raw_ica, preload=True)
threshold = 360e-6 # порог по которому отбросим выраженно артефактные отрезки записи
epochs_ica = epochs_r.drop_bad(reject=dict(eeg=threshold))

In [ ]:
# !pip install scikit-learn # for mne.ICA

In [ ]:
# тренировка модели ICA
ica = mne.preprocessing.ICA(n_components = 20).fit(epochs_ica)

In [ ]:
# визуализируем компоненты
ica.plot_components(inst = epochs_ica)
plt.show()

In [ ]:
# Для воспроизводимости модели модно задать параметр random_state
ica = mne.preprocessing.ICA(n_components = 20, random_state=95).fit(epochs_ica)

In [ ]:
ica.plot_components(inst=epochs_ica) # горизонтальное движение глаз - всегда с одной стороны красное, с другой синее; вертикальные движения глаз - ещё одно

In [ ]:
#посмотреть сигнал компонентов
ica.plot_sources(inst = epochs_ica, start=20)
plt.show()

In [ ]:
# ica.plot_properties(raw)

In [ ]:
#помечаем компоненты которые нам не нравятся, смотрим на них поподробнее
ica.exclude = [2, 4]
ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
ica

In [ ]:
ica.plot_overlay(raw)
plt.show()

In [ ]:
#сделаем еще одну копию исходного файла, из которой уже исключим выбранные артефактные компоненты
reconst_raw = raw.copy()
#применяем модель на копии изначальных данных
ica.apply(reconst_raw)
#смотрим как было
raw.plot(n_channels = len(raw.ch_names), duration=20.0, start=10.0, scalings={'eeg':1e-4})
#смотрим как стало
reconst_raw.plot(n_channels = len(raw.ch_names), duration=20.0, start=10.0, scalings={'eeg':1e-4})
plt.show()

In [ ]:
#делаем перереферирование на усредненный электрод
reconst_raw =reconst_raw.set_eeg_reference(ref_channels='average')

In [ ]:
reconst_raw.save(f'{path}{ID}_RT_clean.fif', overwrite=True)

И в заключение посмотрим, что мы можем сделать есть у нас зарание не даны позиции каналов

In [ ]:
subject='58'
raw = mne.io.read_raw_edf(path + subject + '_RT.edf', preload = True)

In [ ]:
raw

In [ ]:
print(len(raw.ch_names))
raw.ch_names

In [ ]:
# [ch.split()[1].split("-")[0] for ch in raw.ch_names]

In [ ]:
# mne.viz.plot_sensors(raw.info, kind='topomap', show_names=True)

In [ ]:
raw.ch_names

In [ ]:
names_64 = ['Fp1', 'Fpz', 'Fp2', 'F7','F3','Fz','F4','F8','FC5',
 'FC1','FC2','FC6','M1','T7','C3','Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1',
 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'O2',
 'EOG','AF7','AF3','AF4','AF8','F5','F1','F2','F6','FC3','FCz','FC4',
 'C5','C1','C2','C6','CP3','CP4','P5','P1','P2','P6','PO5','PO3','PO4','PO6','FT7','FT8',
 'TP7','TP8', 'PO7', 'PO8', 'Oz']

In [ ]:
raw.rename_channels(dict(zip(raw.ch_names, names_64)))# переиминовываем каналы по словарю
raw.set_channel_types(mapping = {'EOG':'eog'})# задаем тип каналов

ch = mne.channels.make_standard_montage('standard_1005') # устанавливаем стандартный монтаж
raw.set_montage(ch)

In [ ]:
raw.ch_names

In [ ]:
mne.viz.plot_sensors(raw.info, kind='topomap', show_names=True)
plt.show()